In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
#downloading data using sql query
query = """
 SELECT u.user_id,
        u.lt_day,
        
         CASE
             WHEN lt_day>365 THEN 'old'
             ELSE 'new'
         END 
         AS is_new,
         u.age,
         u.os_name,
         CASE
              WHEN u.gender_segment=1 THEN 'female'
              WHEN u.gender_segment=0 THEN'male'
              ELSE 'none'
         END as gender_segment,
        u.cpe_type_name,
        l.country,
        l.city,
        a.title as age_segment,
        t.title as traffic_segment,
        l.title as lifetime_segment,
        u.nps_score,
        CASE
            WHEN u.nps_score >=0 and u.nps_score<=6 THEN 'критики'
            WHEN u.nps_score >=7 and u.nps_score<=8 THEN 'нейтральные'
            ELSE 'сторонники'
        END
        AS npc_group
        
 FROM user as u
 JOIN location as l on u.location_id=l.location_id
  JOIN age_segment as a on u.age_gr_id=a.age_gr_id
 JOIN traffic_segment as t on u.tr_gr_id=t.tr_gr_id
 JOIN lifetime_segment as l on u.lt_gr_id=l.lt_gr_id
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(15)

,user_id,lt_day,is_new,age,os_name,gender_segment,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,npc_group
0,A001A2,2320,old,45.0,ANDROID,female,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,сторонники
1,A001WF,2344,old,53.0,ANDROID,male,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,сторонники
2,A003Q7,467,old,57.0,ANDROID,male,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,сторонники
3,A004TB,4190,old,44.0,IOS,female,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,сторонники
4,A004XT,1163,old,24.0,ANDROID,male,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,сторонники
5,A005O0,5501,old,42.0,ANDROID,female,SMARTPHONE,Россия,Омск,04 35-44,05 5-10,08 36+,6,критики
6,A0061R,1236,old,45.0,ANDROID,male,SMARTPHONE,Россия,Уфа,05 45-54,06 10-15,08 36+,10,сторонники
7,A009KS,313,new,35.0,ANDROID,male,SMARTPHONE,Россия,Москва,04 35-44,13 45-50,05 7-12,10,сторонники
8,A00AES,3238,old,36.0,ANDROID,female,SMARTPHONE,Россия,СанктПетербург,04 35-44,04 1-5,08 36+,10,сторонники
9,A00F70,4479,old,54.0,ANDROID,female,SMARTPHONE,Россия,Волгоград,05 45-54,07 15-20,08 36+,9,сторонники


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

###  Tableau Public dashboard:

https://public.tableau.com/app/profile/german1314/viz/Telecomcompanydash/Dashboard1?publish=yes

### Presentation:

https://1drv.ms/p/s!Av1D_sdJ6zeIhF7GsL6MPGvrl2Tr?e=fymuJe

In [6]:
d = df.assign(total_nps=lambda row: row['npc_group'].map({'сторонники': 1, 'нейтральные': 0, 'критики': -1}))\
  .pivot_table(index='gender_segment', columns='is_new', values='total_nps', aggfunc='mean')
display(d)

is_new,new,old
gender_segment,,
female,0.387000,0.230023
male,0.325246,0.144461
none,0.235374,-0.404594


In [7]:

total_nps = df.assign(total_nps=lambda row: row['npc_group'].map({'сторонники': 1, 'нейтральные': 0, 'критики': -1}))\
  ['total_nps'].mean()
print(f"Total_nps: {total_nps:.1%}")

Total_nps: 21.9%


<hr style="border: 2px solid red;" />